In [1]:
# Step 1: Prepare essential modules: FoldSeekAlignment, FoldSeekAlignmentParser


class FoldSeekAlignment:
    def __init__(self, line):
        fields = line.strip().split('\t')
        
        if len(fields) == 21:
            # for most databases
            self.qseqid = fields[0]
            self.tseqid = fields[1]
            self.pident = float(fields[2])
            self.alnlen = int(fields[3])
            self.mismatch = int(fields[4])
            self.gapopen = int(fields[5])
            self.qstart = int(fields[6])
            self.qend = int(fields[7])
            self.tstart = int(fields[8])
            self.tend = int(fields[9])
            self.evalue = float(fields[10])
            self.bitscore = float(fields[11])
            self.prob = int(fields[12])
            self.qlen = int(fields[13])
            self.tlen = int(fields[14])
            self.qaln = fields[15]
            self.taln = fields[16]
            self.tca = [float(x) for x in fields[17].split(',')]
            self.tseq = fields[18]
            self.ttaxid = fields[19]
            self.ttaxname = fields[20]
        elif len(fields) == 19:
            # for GMGC and MGnify
            self.qseqid = fields[0]
            self.tseqid = fields[1]
            self.pident = float(fields[2])
            self.alnlen = int(fields[3])
            self.mismatch = int(fields[4])
            self.gapopen = int(fields[5])
            self.qstart = int(fields[6])
            self.qend = int(fields[7])
            self.tstart = int(fields[8])
            self.tend = int(fields[9])
            self.evalue = float(fields[10])
            self.bitscore = float(fields[11])
            self.prob = int(fields[12])
            self.qlen = int(fields[13])
            self.tlen = int(fields[14])
            self.qaln = fields[15]
            self.taln = fields[16]
            self.tca = [float(x) for x in fields[17].split(',')]
            self.tseq = fields[18]
            self.ttaxid = None
            self.ttaxname = None
        else:
            raise ValueError("Invalid FoldSeek .m8 line format")

    def __str__(self):
        return f"Query: {self.query_id}, target: {self.pident}, E-value: {self.evalue}, Bit Score: {self.bit_score}"
    
    def print_alignment(self):
        print(f"Query: {self.qaln}")
        print(f"Targt: {self.taln}")



class FoldSeekAlignmentParser:
    def __init__(self, filename):
        self.filename = filename

    def parse(self):
        with open(self.filename, 'r') as f:
            alignments = [FoldSeekAlignment(line) for line in f.readlines()]
        return alignments



def check_reasonability(seq):
    alphabet = set(seq)
    if alphabet - set("ACDEFGHIKLMNPQRSTVWY") != set():
        return False
    return True

In [2]:
# Step 2: input starting point, protected region
folder_path = "EPa23"

# set protected region
protect_start = 9
protect_end = 159

In [3]:
# Step 3: Read and parse the alignment file
folder_path = folder_path

filename_list = [
    folder_path + "/alis_afdb-proteome.m8",
    folder_path + "/alis_afdb-swissprot.m8",
    folder_path + "/alis_afdb50.m8",
    folder_path + "/alis_cath50.m8",
    folder_path + "/alis_gmgcl_id.m8",
    folder_path + "/alis_mgnify_esm30.m8",
    folder_path + "/alis_pdb100.m8"
]

alignments_collection = []
alignments_dbname = [
    "afdb_proteome",
    "afdb_swissprot",
    "afdb50",
    "cath50",
    "gmgcl_id",
    "mgnify_esm30",
    "pdb100"
]

# Parse the FoldSeek result files
for filename in filename_list:
    parser = FoldSeekAlignmentParser(filename)
    alignments = parser.parse()
    alignments_collection.append(alignments)

total_alignments = 0
# print results
for i in range(len(alignments_dbname)):
    print("Database:",alignments_dbname[i],", found alignments:",len(alignments_collection[i]))
    total_alignments += len(alignments_collection[i])


print("=====================================")


# save to fasta
f_out = open(f"{folder_path}/{folder_path}_foldseek.fasta","w")
count = 0

for alignments_index in range(len(alignments_dbname)):
    alignments = alignments_collection[alignments_index]
    alignments_db = alignments_dbname[alignments_index]

    for alignment in alignments:
        # consider protect region
        if alignment.qstart <= protect_start and alignment.qend >= protect_end:
            f_out.write(">" + alignments_db + " " + alignment.tseqid.split(" ")[0] + "\n")
            f_out.write(alignment.tseq + "\n")
            count += 1

print("Total fraction of select alignments:",count/total_alignments)
print("Total number of select alignments:",count)

f_out.close()


Database: afdb_proteome , found alignments: 9
Database: afdb_swissprot , found alignments: 10
Database: afdb50 , found alignments: 993
Database: cath50 , found alignments: 14
Database: gmgcl_id , found alignments: 66
Database: mgnify_esm30 , found alignments: 998
Database: pdb100 , found alignments: 74
Total fraction of select alignments: 0.7980591497227357
Total number of select alignments: 1727
